In [1]:
import os
import glob
from collections import OrderedDict,defaultdict
import pipes
import pprint
import tempfile
import re

In [2]:
root_dir = 'K562_Chen_data'

In [3]:
####Fixed length 1e6 = 100 blocks. 
####Fixed length 5e5 = 200 blocks. 
Fixed_length = 1.5e6

In [4]:
def binary_locate(array,ele,start,end):
    middle = (end+start)//2
    if(end-start>1):
        if(array[middle]<ele):
            return binary_locate(array,ele,middle,end)
        elif(array[middle+1]>ele):
            return binary_locate(array,ele,start,middle)
        else:
            return middle,middle+1
    else:
        return middle,middle+1

In [5]:
p = pipes.Template()
p.append("zcat", '--')
Gene_Loc = defaultdict(list)
f = p.open('/home/longy/cnda/ensembl/Homo_sapiens.GRCh38.103.gtf.gz', 'r')

#f = p.open('/home/longy/cnda/ensembl/Mus_musculus.GRCm38.102.gtf.gz', 'r')
for line in f.readlines():
    if re.match('^#',line):
        continue
    line = line.rstrip('\n')
    data = line.split('\t')
    if(data[2] == 'gene'):
        Gene_Loc['chr'+data[0]+data[6]].append(int(data[3]))
        Gene_Loc['chr'+data[0]+data[6]].append(int(data[4]))

In [6]:
import sys
sys.setrecursionlimit(10000)
array = Gene_Loc['chr1+']
binary_locate(array,46104790,0,3505)

(1523, 1524)

In [7]:
total = 0
files = glob.glob(root_dir+"/*.txt")
info = open(root_dir+'/Blocks/info.txt','w')
info.write('ID\tstart\tend\tlength\tsites_num\n')
for file in files:
    num_lines = sum(1 for line in open(file))
    num_blocks = round(num_lines/Fixed_length)
    len_blocks = num_lines/num_blocks
    
    f= open(file,'r')
    pre_pos = 0
    dict_line = OrderedDict()
    #dict_line = dict()
    count = 0
    separate_num=0
    save_block = 0
    start = 0
    
    for i, line in enumerate(f):
        line = line.rstrip('\n')
        if(i==0):
            continue ###Skip header
        chromosome,pos,strand = line.split('\t')[0].split(':')
        pos = int(pos)
        count += 1
        if(i==1):
            start=pos
        if(count>len_blocks and pos-pre_pos>1000 and save_block<num_blocks-1):
            array = Gene_Loc[chromosome+strand]
            index1,index2 = binary_locate(array,pre_pos,0,len(array)-1)
            if(index1%2==1 and index2%2==0):
                ww = open(root_dir+'/Blocks/%s_%s_%d'%(chromosome,strand,save_block),'w')
                print('writing file '+root_dir+'/Blocks/%s_%s_%d_%d-%d'%(chromosome,strand,save_block,start,pre_pos))
                info.write('%s_%s_%d\t%d\t%d\t%d\t%d\n'%(chromosome,strand,save_block,start,pre_pos,pre_pos-start,count))
                #print('%s_%s_%d\t%d\t%d\t%d\t%d\n'%(chromosome,strand,save_block,start,pre_pos,pre_pos-start,count))
                for key,val in dict_line.items():
                    ww.write('%s\n'%val)
                ww.close()
                dict_line.clear()
                count = 0
                save_block += 1
                start  = pos
            
        
        dict_line[pos] = line
        pre_pos = pos
        
    out = open(root_dir+'/Blocks/%s_%s_%d'%(chromosome,strand,save_block),'w')
    print('writing file '+root_dir+'/Blocks/%s_%s_%d_%d-%d'%(chromosome,strand,save_block,start,pre_pos))
    info.write('%s_%s_%d\t%d\t%d\t%d\t%d\n'%(chromosome,strand,save_block,start,pre_pos,pre_pos-start,count))
    for key,val in dict_line.items():
        out.write('%s\n'%(val))
    out.close()

info.close()

writing file K562_Chen_data/Blocks/chr18_-_0_47966-36129805
writing file K562_Chen_data/Blocks/chr18_-_1_36154863-80247429
writing file K562_Chen_data/Blocks/chr20_-_0_214924-30631157
writing file K562_Chen_data/Blocks/chr20_-_1_30633372-46035582
writing file K562_Chen_data/Blocks/chr20_-_2_46043584-64333153
writing file K562_Chen_data/Blocks/chr20_+_0_130003-29489322
writing file K562_Chen_data/Blocks/chr20_+_1_29490793-45083659
writing file K562_Chen_data/Blocks/chr20_+_2_45085356-64329721
writing file K562_Chen_data/Blocks/chr2_-_0_39326-27393724
writing file K562_Chen_data/Blocks/chr2_-_1_27397702-54272101
writing file K562_Chen_data/Blocks/chr2_-_2_54274069-73816760
writing file K562_Chen_data/Blocks/chr2_-_3_73820320-99156435
writing file K562_Chen_data/Blocks/chr2_-_4_99159654-127799050
writing file K562_Chen_data/Blocks/chr2_-_5_127801322-159616589
writing file K562_Chen_data/Blocks/chr2_-_6_159635145-185421330
writing file K562_Chen_data/Blocks/chr2_-_7_185450138-212429415
wri

KeyboardInterrupt: 

In [12]:
text_file = open("STAR/HepG2_Control/Signal.Unique.str1.out.chr10.wig", "r")
lines = text_file.readlines()
text_file.close()
print(len(lines))

8688341


In [14]:
import sys
sys.path.insert(0, '/home/longy/project/python_lib')
from TrimmedMean import TrimmedMean

In [32]:
fa_file = open("/ibex/scratch/longy/cnda/ensembl/oneLine/hg38.22.fa","r")

In [33]:
line = fa_file.readline()

In [36]:
a = line[300:400]

In [37]:
a

'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'

In [38]:
fa_file.close()

In [46]:
a = 'afta'
a = a.upper()
print(a)

AFTA


In [67]:
import re
string = 'TTTCTGCTGTAGAGTCTGGCAT'
match = re.findall('GT',string)

In [68]:
match

['GT', 'GT']

In [58]:
match.span()

(9, 12)

In [87]:
string = 'fd#faatag.gz.fafa'

In [88]:
if re.findall('^#',string):
    print('find')
else:
    print('not found')

not found


In [108]:
import random
random.random()

0.06320255234152405

In [114]:
coverage = ['3','5','45','34']
[float(coverage[i]) for i in range(int(len(coverage)/2))]

[3.0, 5.0]

In [116]:
import math
round(3.5)

4

In [117]:
a = '-3'
a = float(a)

In [119]:
round(4.5)

4

In [122]:
a = 'Origin'
'Origin' in a

True